In [1]:
import numpy as np
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
# from sklearn.ensembl import RandomForestClassifier
import warnings
import gc


- import MetabolitesFromInChI.csv

In [7]:
df = pd.read_csv("https://raw.githubusercontent.com/meidelien/Metabolic-network-layout-using-biochemical-coordinates/main/Notebooks/MetabolitesFromInChIKey.csv")

# Creating new df with descriptors
- Iterating over InChI's from MetabolitesFromInChI df 
- Storing DescriptorCalculator output to descrs list
- Creating molDes df & appending Descriptors to each column


In [ ]:
mol_list = []

for i in df.InChI:
    mol = Chem.MolFromInchi(i)
    mol_list.append(mol)


desc_list=[i[0] for i in Descriptors._descList]
descriptor = MoleculeDescriptors.MolecularDescriptorCalculator(desc_list)

descrs =  []

for i in range(len(mol_list)):
    descrs.append(descriptor.CalcDescriptors(mol_list[i]))
molDes = pd.DataFrame(descrs, columns= desc_list)
molDes.head(5)

- Make csv cus reasons

In [22]:
molDes.to_csv("chemical_properties.csv", index = False)

In [32]:
data1 = pd.read_csv("MetabolitesFromInChIKeyWIP.csv")
data2 = pd.read_csv("chemical_properties.csv")


- Concatatenating MetabolitesFromInChIKey & chemical_properties.csv

In [39]:
result = pd.concat([data1, data2], axis=1).reindex(data1.index)

In [40]:
result.to_csv("Chemical_descriptors.csv", index = False)